# Import thư viện

In [101]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re
import os
import sys
import glob
import shutil
from pathlib import Path
import re

In [102]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [103]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

# Thông tin phim

In [104]:
# Mở url
url = 'https://www.rottentomatoes.com/m/limo'
# url = 'https://www.rottentomatoes.com/m/beg1994'
# url = 'https://www.rottentomatoes.com/m/antman'
driver.get(url)

In [105]:
# Đợi cho đến khi url load xong
element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

In [106]:
# Tiêu đề phim
title = soup.select('h1[class="scoreboard__title"]')[0].text
title

'Limo'

In [107]:
# Điểm đánh giá của nhà phê bình
tomatometer = soup.select('score-board')[0].attrs['tomatometerscore']
tomatometer

''

In [108]:
# Điểm đánh giá của khán giả
audience_score = soup.select('score-board')[0].attrs['audiencescore']
audience_score

''

In [109]:
# Số lượt đánh giá của nhà phê bình
num_tomatometer = soup.select('a[slot="critics-count"]')[0].text.strip()
num_tomatometer

'0 Reviews'

In [110]:
# Số lượt đánh giá của khán giả
num_audience_score = soup.select('a[slot="audience-count"]')[0].text.strip()
num_audience_score

'0 Ratings'

In [111]:
# The Tomatometer score
tomatometer_state = soup.select('score-board')[0].attrs['tomatometerstate']
tomatometer_state

''

In [112]:
# The Audience Score
audience_state = soup.select('score-board')[0].attrs['audiencestate']
audience_state

''

In [113]:
# Thông tin phim
list_name_info = soup.select('div[class="meta-label subtle"]')
list_info = soup.select('div[data-qa="movie-info-item-value"]')

for num_info in range(len(list_name_info)):
    name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    print(name_info + info)

GenreAdventure
Original LanguageDutch
DirectorGuy Goossens
ProducerGuy Goossens, Mark Punt
WriterMark Punt
Runtime1h 36m


In [114]:
# Tóm tắt nội dụng phim
synopsis = soup.select('div[id="movieSynopsis"]')[0].text.strip()
synopsis

'Four girls visit a mysterious village.'

In [115]:
# Diễn viên
list_raw_cast = soup.select('span[class="characters subtle smaller"]')
list_cast = []

for c in list_raw_cast:
    list_cast.append(c.attrs['title'])

cast = ', '.join(list_cast)
cast

'Jan Decleir, Jack Wouterse, Jaap Spijkers, Inge Ipenburg, Meike Hurts, Fleur Minjon, Anouk van Schie, Monique de Waal, Manuel Broekman, Guy Goossens, Mark Punt, Guy Goossens, Mark Punt, Tuomas Kantelinen, Gerd Schelfhout, Alain Dessauvage, Philippe Ravoet, Janusz Gosschalk, Saida van der Reijd, Kes Bonnet, Vyarka Sirkova'

# Reviews phim

### Reviews của nhà phê bình

In [116]:
# Chuyển đến url review
driver.get(url + '/reviews')

element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

In [117]:
# Check có review của nhà phê bình không ?
check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

if check_critic_review != '':
    list_critic_review = []
    
    for rv in soup_critic_review.select('p[class="review-text"]'):
        list_critic_review.append(rv.text)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
            soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

            for rv in soup_critic_review.select('p[class="review-text"]'):
                list_critic_review.append(rv.text.replace('  ', ''))
            
        except:
            break
    critic_review = '<>'.join(list_critic_review)
    print(critic_review)
elif check_critic_review == '':
    critic_review = 'Không có review của nhà phê bình'
    print(critic_review)

Không có review của nhà phê bình


### Review của khán giả

In [118]:
# Chuyển đến url review của khán giả
driver.get(url + '/reviews?type=user')

element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

In [119]:
# Check có review của khán giả không ?
check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

if len(check_audience_review) != 0:
    list_audience_review = []
    
    for rv in soup_audience_review.select('p[data-qa="review-text"]'):
        list_audience_review.append(rv.text)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
            soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

            for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                list_audience_review.append(rv.text)
            
        except:
            break
    audience_review = '<>'.join(list_audience_review)
    print(audience_review)
else:
    audience_review = 'Không có review của khán giả'
    print(audience_review)

Không có review của khán giả
